# Object Detection

## Imports

In [4]:
import torch
import pandas as pd
import os
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt

# PyTorch helper functions
from . import pytorchHelperFunctions.transforms.py as T
from "engine.py" import train_one_epoch, evaluate
import "utils.py"

SyntaxError: invalid syntax (318780664.py, line 7)

## Global Variables

In [ ]:
TRAIN_LABELS_PATH = './vinbigdata/labels/train'
VAL_LABELS_PATH = './vinbigdata/labels/val'
TRAIN_IMAGES_PATH = './vinbigdata/images/train' #12000
VAL_IMAGES_PATH = './vinbigdata/images/val' #3000
MODEL_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/models/vehicle_detection_model.pth'

INFERENCE_IMG_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/data/example.jpg"
External_DIR = '../input/vinbigdata-512-image-dataset/vinbigdata/train' # 15000
os.makedirs(TRAIN_LABELS_PATH, exist_ok = True)
os.makedirs(VAL_LABELS_PATH, exist_ok = True)
os.makedirs(TRAIN_IMAGES_PATH, exist_ok = True)
os.makedirs(VAL_IMAGES_PATH, exist_ok = True)
num_classes = 51

## Dataset Preprocessing

In [ ]:
df = pd.read_csv('../input/vinbigdata-512-image-dataset/vinbigdata/train.csv')
df.head()

## PyTorch Dataset Module

In [ ]:
class VehSatDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.labels = list(sorted(os.listdir(os.path.join(root, "labels"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        label_path = os.path.join(self.root, "labels", self.labels[idx])
        img = Image.open(img_path)#.convert("RGB")

        labels = pd.read_pickle(label_path)

        # get bounding box coordinates for each mask
        num_objects = len(labels)
        object_classes = []
        boxes = []
        for i in range(num_objects):
            img_label = labels.iloc[i,:]
            object_class = img_label[0]
            xmin = img_label[1]
            xmax = img_label[2]
            ymin = img_label[3]
            ymax = img_label[4]
            object_classes.append([object_class])
            boxes.append([xmin, ymin, xmax, ymax])
        

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        object_classes = torch.as_tensor(object_classes, dtype=torch.float32)

        image_id = torch.tensor([idx])

        target = {}
        target["boxes"] = boxes
        target["labels"] = object_classes
        target["image_id"] = image_id

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

## PyTorch Model

In [ ]:
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', autoshape=False, pretrained=True, num_classes=num_classes)

In [ ]:
# Change this when using a different model
model = yolov5

## Data Augmentation

In [3]:
def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

## Forward Pass (Before Training)

In [ ]:
dataset = VehSatDataset('VehSat', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=2, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)

# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

## Training

In [ ]:
def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = VehSatDataset('PennFudanPed', get_transform(train=True))
    dataset_test = VehSatDataset('PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("Training Finished!")

## Save Model

In [ ]:
torch.save(model, MODEL_PATH)

## Model Inference

### Load Model and Image

In [ ]:
model = torch.load(MODEL_PATH)

img = INFERENCE_IMG_PATH

### Perform Inference

In [ ]:
result = model(img)
fig, ax = plt.subplots(figsize=(16, 12))
ax.imshow(result.render()[0])
plt.show()

In [ ]:
df = result.pandas().xyxy[0]  # img1 predictions (pandas)
df.head()

In [ ]:
no_of_detections = len(df.index)
print("Detected {} vehicles".format(no_of_detections))

##